In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random

In [2]:
all_a = []
limit = 30
for i in tqdm(range(0, 100000, limit)):
    if i == 0:
        i = ''
    else:
        i = f'/+{i}'
    url = f'https://forum.lowyat.net/SeriousKopitiam{i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and a_.get('href').startswith('/topic')]
    a = list(set([a_ for a_ in a if len(a_.split('/')) == 3]))
    if not len(a):
        break
    all_a.extend(a)

 54%|█████████████████████▏                 | 1807/3334 [26:09<22:06,  1.15it/s]


In [3]:
all_a = list(set(all_a))

In [4]:
len(all_a)

54149

In [6]:
with open('SeriousKopitiam-topics.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    try:
        r_ = requests.get(url)
        soup = BeautifulSoup(r_.content, "lxml")
        divs = soup.find_all('div', {'class': 'post_text'})
        for div in divs:
            t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
            if len(t):
                texts.append(t)
    except Exception as e:
        pass
    return texts

In [8]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for topic in tqdm(sorted(all_a)):
    
    base_dir = topic[1:].replace('/', '-')
    os.makedirs(base_dir, exist_ok = True)
    os.makedirs('done-SeriousKopitiam', exist_ok = True)
    
    base_dir_filename = os.path.join('done-SeriousKopitiam', f'{base_dir}.json')
    
    if os.path.exists(base_dir_filename):
        continue
    
    need_break = False
    last_texts = []
    for i in range(0, 100000, per_batch):
        filename = os.path.join(base_dir, f'{i}.json')
        if os.path.exists(filename):
            continue
        aranged = np.arange(i, i + per_batch, limit)
        urls = []
        for a_ in aranged:
            if a_ == 0:
                a_ = ''
            else:
                a_ = f'/+{a_}'
            url = f'https://forum.lowyat.net/{topic}{a_}'
            urls.append(url)
        
        texts = []
        
        with ThreadPoolExecutor(max_workers=worker_size) as executor:
            futures = {executor.submit(get_text, item): item for item in urls}
            
            for future in as_completed(futures):
                r = future.result()
                texts.extend(r)
                if not len(r):
                    need_break = True
                    
                s = set(texts)
                if len(set(last_texts) & s) == len(s):
                    need_break = True
                    
                last_texts = texts[:]
        
        with open(filename, 'w') as fopen:
            json.dump(texts, fopen)
            
        time.sleep(random.uniform(0.5, 1.5))
        
        if need_break:
            break
    
    with open(base_dir_filename, 'w') as fopen:
        json.dump({'status': True}, fopen) 

  0%|▏                                   | 264/54149 [19:54<92:15:03,  6.16s/it]encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
  7%|██▏                              | 3569/54149 [4:35:58<60:20:41,  4.30s/it]encoding error : input conversion failed due to input error, bytes 0x98 0x85 0xE2 0x98
encoding error : input conversion failed due to input error, bytes 0x98 0x85 0xE2 0x98
encoding error : input conversion failed due to input error, bytes 0x98 0x85 0xE2 0x98
 14%|████▎                           | 7312/54149 [10:04:05<54:30:48,  4.19s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, byt

encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
 93%|█████████████████████████████▋  | 50158/54149 [73:57:14<5:42:20,  5.15s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0xEF 0xBF 0xBD
encoding error : input conversion failed due to input error, bytes 0x9D 0xEF 0xBF 0xBD
encoding error : input conversion failed due to input error, bytes 0x9D 0xEF 0xBF 0xBD
 93%|█████████████████████████████▋  | 50226/54149 [74:02:52<4:32:31,  4.17s/it]encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE6 0x84
 93%|█████████████████████████████▋  | 50304/54149 [74:09

In [9]:
!mkdir SeriousKopitiam

In [10]:
!mv topic-* SeriousKopitiam

In [11]:
!du -hs SeriousKopitiam

1.1G	SeriousKopitiam


In [13]:
# !zip -r SeriousKopitiam.zip SeriousKopitiam

In [14]:
from glob import glob
texts = []

files = glob('SeriousKopitiam/topic-*/*.json')
for f in tqdm(files):
    with open(f) as fopen:
        texts.extend(json.load(fopen))

100%|████████████████████████████████| 138032/138032 [00:02<00:00, 61131.88it/s]


In [15]:
texts = list(set(texts))
len(texts)

991889

In [17]:
with open('SeriousKopitiam.json', 'w') as fopen:
    json.dump(texts, fopen)